In [3]:
import pickle
import pickle as pkl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal as sig
from scipy.io import loadmat, savemat
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
#get all functions 
import sys, os
code_path = os.path.dirname('/mnt/leif/littlab/users/aguilac/Interictal_Spike_Analysis/HUMAN/working_feat_extract_code/functions/')
sys.path.append(code_path)
from ied_fx_v3 import *

In [2]:
json_labels = '/Users/carlosaguila/PycharmProjects/CNT_Interictal_Spikes/Patient/pt_database/CNT_iEEG_BIDS/sub-RID0179/derivatives/ieeg_recon/module3/sub-RID0179_ses-research3T_space-T00mri_atlas-DKTantspynet_radius-2_desc-vox_coordinates.json'

In [3]:
workinglabels = pd.read_json(json_labels, lines=True)

FileNotFoundError: File /Users/carlosaguila/PycharmProjects/CNT_Interictal_Spikes/Patient/pt_database/CNT_iEEG_BIDS/sub-RID0179/derivatives/ieeg_recon/module3/sub-RID0179_ses-research3T_space-T00mri_atlas-DKTantspynet_radius-2_desc-vox_coordinates.json does not exist

In [ ]:
empty = (workinglabels[workinglabels['label'] == 'EmptyLabel'])
display(empty)

,name,x,y,z,index,label,labels_sorted,percent_assigned
2,LAT3,66.5970,137.4260,26.3088,0,EmptyLabel,"[EmptyLabel, left inferior temporal ]","[0.7272727273, 0.2727272727]"
3,LAT4,57.2303,134.0920,30.9649,0,EmptyLabel,"[EmptyLabel, left inferior temporal ]","[0.6666666667000001, 0.33333333330000003]"
7,LDA4,64.2987,136.7100,40.5375,0,EmptyLabel,"[EmptyLabel, left fusiform ]","[0.9696969697000001, 0.030303030300000002]"
8,LDH1,74.5979,126.0560,44.7521,0,EmptyLabel,"[EmptyLabel, left parahippocampal ]","[0.7575757576000001, 0.24242424240000002]"
10,LDH3,64.1822,125.8690,43.9083,0,EmptyLabel,"[EmptyLabel, left fusiform ]","[0.6060606061, 0.3939393939]"
11,LDH4,58.7263,125.7710,43.4664,0,EmptyLabel,"[EmptyLabel, left fusiform , left inferior t...","[0.3636363636, 0.33333333330000003, 0.303030303]"
12,LPT1,67.1344,106.0680,48.2641,0,EmptyLabel,"[EmptyLabel, left fusiform ]","[0.7272727273, 0.2727272727]"
14,LPT3,54.8546,118.4140,38.0928,0,EmptyLabel,"[EmptyLabel, left inferior temporal ]","[0.5757575758, 0.4242424242]"
16,RDA1,132.1970,145.9820,45.0470,0,EmptyLabel,[EmptyLabel],[1.0]
17,RDA2,134.7750,141.4250,45.0775,0,EmptyLabel,[EmptyLabel],[1.0]


In [ ]:
def unnesting(df, explode, axis):
    if axis==1:
        idx = df.index.repeat(df[explode[0]].str.len())
        df1 = pd.concat([
            pd.DataFrame({x: np.concatenate(df[x].values)}) for x in explode], axis=1)
        df1.index = idx

        return df1.join(df.drop(explode, 1), how='right')
    else :
        df1 = pd.concat([
                         pd.DataFrame(df[x].tolist(), index=df.index).add_prefix(x) for x in explode], axis=1)
        return df1.join(df.drop(explode, 1), how='right')

In [ ]:
empty = unnesting(empty, ['labels_sorted', 'percent_assigned'], axis=0)
display(empty)

,labels_sorted0,labels_sorted1,labels_sorted2,percent_assigned0,percent_assigned1,percent_assigned2,name,x,y,z,index,label
2,EmptyLabel,left inferior temporal,None,0.727273,0.272727,NaN,LAT3,66.5970,137.4260,26.3088,0,EmptyLabel
3,EmptyLabel,left inferior temporal,None,0.666667,0.333333,NaN,LAT4,57.2303,134.0920,30.9649,0,EmptyLabel
7,EmptyLabel,left fusiform,None,0.969697,0.030303,NaN,LDA4,64.2987,136.7100,40.5375,0,EmptyLabel
8,EmptyLabel,left parahippocampal,None,0.757576,0.242424,NaN,LDH1,74.5979,126.0560,44.7521,0,EmptyLabel
10,EmptyLabel,left fusiform,None,0.606061,0.393939,NaN,LDH3,64.1822,125.8690,43.9083,0,EmptyLabel
11,EmptyLabel,left fusiform,left inferior temporal,0.363636,0.333333,0.303030,LDH4,58.7263,125.7710,43.4664,0,EmptyLabel
12,EmptyLabel,left fusiform,None,0.727273,0.272727,NaN,LPT1,67.1344,106.0680,48.2641,0,EmptyLabel
14,EmptyLabel,left inferior temporal,None,0.575758,0.424242,NaN,LPT3,54.8546,118.4140,38.0928,0,EmptyLabel
16,EmptyLabel,None,None,1.000000,NaN,NaN,RDA1,132.1970,145.9820,45.0470,0,EmptyLabel
17,EmptyLabel,None,None,1.000000,NaN,NaN,RDA2,134.7750,141.4250,45.0775,0,EmptyLabel


In [ ]:
empty = empty[np.isnan(empty['percent_assigned1']) == False]

In [ ]:
display(empty)

,labels_sorted0,labels_sorted1,labels_sorted2,percent_assigned0,percent_assigned1,percent_assigned2,name,x,y,z,index,label
2,EmptyLabel,left inferior temporal,None,0.727273,0.272727,NaN,LAT3,66.5970,137.4260,26.3088,0,EmptyLabel
3,EmptyLabel,left inferior temporal,None,0.666667,0.333333,NaN,LAT4,57.2303,134.0920,30.9649,0,EmptyLabel
7,EmptyLabel,left fusiform,None,0.969697,0.030303,NaN,LDA4,64.2987,136.7100,40.5375,0,EmptyLabel
8,EmptyLabel,left parahippocampal,None,0.757576,0.242424,NaN,LDH1,74.5979,126.0560,44.7521,0,EmptyLabel
10,EmptyLabel,left fusiform,None,0.606061,0.393939,NaN,LDH3,64.1822,125.8690,43.9083,0,EmptyLabel
11,EmptyLabel,left fusiform,left inferior temporal,0.363636,0.333333,0.303030,LDH4,58.7263,125.7710,43.4664,0,EmptyLabel
12,EmptyLabel,left fusiform,None,0.727273,0.272727,NaN,LPT1,67.1344,106.0680,48.2641,0,EmptyLabel
14,EmptyLabel,left inferior temporal,None,0.575758,0.424242,NaN,LPT3,54.8546,118.4140,38.0928,0,EmptyLabel
18,EmptyLabel,right middle temporal,right superior temporal,0.909091,0.060606,0.030303,RDA3,137.7450,137.0660,46.2329,0,EmptyLabel
19,EmptyLabel,right middle temporal,None,0.969697,0.030303,NaN,RDA4,140.7160,132.7070,47.3884,0,EmptyLabel


In [ ]:
changed = empty[empty['percent_assigned1'] > 0.25]


In [ ]:
display(changed)

NameError: name 'changed' is not defined

In [ ]:
data_directory = '/Users/carlosaguila/PycharmProjects/CNT_Interictal_Spikes/Patient/pt_database'
ptnames = ['HUP105','HUP106','HUP107','HUP111']
spike, brain_df, ids = load_ptall(ptnames[3],data_directory)


In [ ]:
pd.set_option('display.max_rows', 200)
brain_df['name'] = brain_df['name'].str.replace('-CAR','')
display(brain_df.merge(changed[['labels_sorted1', 'percent_assigned1']], left_on=brain_df['name'], right_on=changed['name'], how = 'left', indicator=True))

,key_0,Unnamed: 0,name,x,y,z,index,label,labels_sorted1,percent_assigned1,_merge
0,LAT1,0,LAT1,80.8064,149.0920,32.5711,1009.0,left inferior temporal,NaN,NaN,left_only
1,LAT2,1,LAT2,74.9307,142.1780,27.1623,1009.0,left inferior temporal,NaN,NaN,left_only
2,LAT3,2,LAT3,66.5970,137.4260,26.3088,0.0,EmptyLabel,left inferior temporal,0.272727,both
3,LAT4,3,LAT4,57.2303,134.0920,30.9649,0.0,EmptyLabel,left inferior temporal,0.333333,both
4,LDA1,4,LDA1,79.6568,139.6700,41.4514,1006.0,left entorhinal,NaN,NaN,left_only
5,LDA2,5,LDA2,74.7373,138.6230,40.7986,1006.0,left entorhinal,NaN,NaN,left_only
6,LDA3,6,LDA3,69.2384,137.2780,41.0649,1006.0,left entorhinal,NaN,NaN,left_only
7,LDA4,7,LDA4,64.2987,136.7100,40.5375,0.0,EmptyLabel,NaN,NaN,left_only
8,LDH1,8,LDH1,74.5979,126.0560,44.7521,0.0,EmptyLabel,NaN,NaN,left_only
9,LDH2,9,LDH2,69.6380,125.9670,44.3503,1016.0,left parahippocampal,NaN,NaN,left_only


In [ ]:
print(ids)

['HUP111', 'sub-RID0179']


In [ ]:
def unnesting(df, explode, axis):
    '''
    code that expands lists in a column in a dataframe.
    '''
    if axis==1:
        idx = df.index.repeat(df[explode[0]].str.len())
        df1 = pd.concat([
            pd.DataFrame({x: np.concatenate(df[x].values)}) for x in explode], axis=1)
        df1.index = idx

        return df1.join(df.drop(explode, 1), how='right')
    else :
        df1 = pd.concat([
                         pd.DataFrame(df[x].tolist(), index=df.index).add_prefix(x) for x in explode], axis=1)
        return df1.join(df.drop(explode, 1), how='right')

def load_rid_forjson(ptname, data_directory):
    ptids = pd.read_csv(data_directory[0] + '/pt_data/all_ptids.csv')
    rid = ptids['r_id'].loc[ptids['hup_id'] == ptname].astype('string')
    rid = np.array(rid)

    if path.exists(data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-research3T_space-T00mri_atlas-DKTantspynet_radius-2_desc-vox_coordinates.csv'.format(rid[0],rid[0])) == True:
        dkt_directory = data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-research3T_space-T00mri_atlas-DKTantspynet_radius-2_desc-vox_coordinates.csv'.format(rid[0],rid[0])
    if path.exists(data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-implant01_space-T00mri_atlas-DKTantspynet_radius-2_desc-vox_coordinates.csv'.format(rid[0],rid[0])) == True:
        dkt_directory = data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-implant01_space-T00mri_atlas-DKTantspynet_radius-2_desc-vox_coordinates.csv'.format(rid[0],rid[0])
    if path.exists(data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-clinical01_space-T00mri_atlas-DKTantspynet_radius-2_desc-vox_coordinates.csv'.format(rid[0],rid[0])) == True:
        dkt_directory = data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-clinical01_space-T00mri_atlas-DKTantspynet_radius-2_desc-vox_coordinates.csv'.format(rid[0],rid[0])

    brain_df = pd.read_csv(dkt_directory)
    brain_df['name'] = brain_df['name'].astype(str) + '-CAR'
    return rid[0], brain_df
        
def label_fix(pt, data_directory, threshold = 0.20):
    '''
    label_fix reassigns labels overlapping brain regions to "empty labels" in our DKTantspynet output from IEEG_recon
    input:  pt - name of patient. example: 'HUP100' 
            data_directory - directory containing CNT_iEEG_BIGS folder. (must end in '/')
            threshold - arbitrary threshold that r=2mm surround of electrode must overlap with a brain region. default: threshold = 25%, Brain region has a 25% or more overlap.
    output: relabeled_df - a dataframe that contains 2 extra columns showing the second most overlapping brain region and the percent of overlap. 
    '''

    rid, brain_df = load_rid_forjson(pt, data_directory)
    if path.exists(data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-research3T_space-T00mri_atlas-DKTantspynet_radius-2_desc-vox_coordinates.json'.format(rid,rid)) == True:
        json_labels = data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-research3T_space-T00mri_atlas-DKTantspynet_radius-2_desc-vox_coordinates.json'.format(rid,rid)
    if path.exists(data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-implant01_space-T00mri_atlas-DKTantspynet_radius-2_desc-vox_coordinates.json'.format(rid,rid)) == True:
        json_labels = data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-implant01_space-T00mri_atlas-DKTantspynet_radius-2_desc-vox_coordinates.json'.format(rid,rid)
    if path.exists(data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-clinical01_space-T00mri_atlas-DKTantspynet_radius-2_desc-vox_coordinates.json'.format(rid,rid)) == True:
        json_labels = data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-clinical01_space-T00mri_atlas-DKTantspynet_radius-2_desc-vox_coordinates.json'.format(rid,rid)
        
    workinglabels = pd.read_json(json_labels, lines=True)
    empty = (workinglabels[workinglabels['label'] == 'EmptyLabel'])
    empty = unnesting(empty, ['labels_sorted', 'percent_assigned'], axis=0)
    empty = empty[np.isnan(empty['percent_assigned1']) == False]
    changed = empty[empty['percent_assigned1'] >= threshold]
    
    brain_df['name'] = brain_df['name'].str.replace('-CAR','')
    relabeled_df = brain_df.merge(changed[['labels_sorted1', 'percent_assigned1']], left_on=brain_df['name'], right_on=changed['name'], how = 'left', indicator=True)   
    relabeled_df['final_label'] = relabeled_df['labels_sorted1'].fillna(relabeled_df['label'])
    relabeled_df['name'] = relabeled_df['name'].astype(str) + '-CAR' #added for this version for our analysis

    return relabeled_df

## Follow up:

- Add grey matter tracker to force labels

In [75]:
data_directory = ['/mnt/leif/littlab/users/aguilac/Projects/FC_toolbox/results/mat_output_v2', '/mnt/leif/littlab/data/Human_Data']
rid, brain_df = load_rid_forjson("HUP142", data_directory)
display(brain_df)

,Unnamed: 0,name,x,y,z,index,label
0,0,LDA1-CAR,85.1479,129.4360,74.9584,0.0,EmptyLabel
1,1,LDA2-CAR,80.4507,128.0110,73.9757,17.0,left hippocampus
2,2,LDA3-CAR,75.1392,127.7570,73.8188,17.0,left hippocampus
3,3,LDA4-CAR,70.4420,126.3330,72.8361,17.0,left hippocampus
4,4,LDA5-CAR,65.1305,126.0790,72.6792,0.0,EmptyLabel
...,...,...,...,...,...,...,...
111,111,LGR60-CAR,30.5581,117.5990,83.7255,0.0,EmptyLabel
112,112,LGR61-CAR,29.8546,107.8410,87.0782,0.0,EmptyLabel
113,113,LGR62-CAR,31.2757,98.1849,90.4937,1015.0,left middle temporal
114,114,LGR63-CAR,31.2763,87.3494,89.9787,1015.0,left middle temporal


In [76]:
if path.exists(data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-research3T_space-T00mri_atlas-DKTantspynet_radius-2_desc-vox_coordinates.json'.format(rid,rid)) == True:
    json_labels = data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-research3T_space-T00mri_atlas-DKTantspynet_radius-2_desc-vox_coordinates.json'.format(rid,rid)
elif path.exists(data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-implant01_space-T00mri_atlas-DKTantspynet_radius-2_desc-vox_coordinates.json'.format(rid,rid)) == True:
    json_labels = data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-implant01_space-T00mri_atlas-DKTantspynet_radius-2_desc-vox_coordinates.json'.format(rid,rid)
elif path.exists(data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-clinical01_space-T00mri_atlas-DKTantspynet_radius-2_desc-vox_coordinates.json'.format(rid,rid)) == True:
    json_labels = data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-clinical01_space-T00mri_atlas-DKTantspynet_radius-2_desc-vox_coordinates.json'.format(rid,rid)

if path.exists(data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-research3T_space-T00mri_atlas-atropos_radius-2_desc-vox_coordinates.json'.format(rid,rid)) == True:
    at_json_labels = data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-research3T_space-T00mri_atlas-atropos_radius-2_desc-vox_coordinates.json'.format(rid,rid)
elif path.exists(data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-implant01_space-T00mri_atlas-atropos_radius-2_desc-vox_coordinates.json'.format(rid,rid)) == True:
    at_json_labels = data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-implant01_space-T00mri_atlas-atropos_radius-2_desc-vox_coordinates.json'.format(rid,rid)
elif path.exists(data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-clinical01_space-T00mri_atlas-atropos_radius-2_desc-vox_coordinates.json'.format(rid,rid)) == True:
    at_json_labels = data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-clinical01_space-T00mri_atlas-atropos_radius-2_desc-vox_coordinates.json'.format(rid,rid)

atropos_labels = pd.read_json(at_json_labels, lines = True)    
#at_threshold = 0.1

workinglabels = pd.read_json(json_labels, lines=True)
threshold = 0.2

empty = (workinglabels[workinglabels['label'] == 'EmptyLabel'])
empty = unnesting(empty, ['labels_sorted', 'percent_assigned'], axis=0)
empty = empty[np.isnan(empty['percent_assigned1']) == False]
changed = empty[empty['percent_assigned1'] >= threshold]

brain_df['name'] = brain_df['name'].str.replace('-CAR','')
relabeled_df = brain_df.merge(changed[['labels_sorted1', 'percent_assigned1']], left_on=brain_df['name'], right_on=changed['name'], how = 'left', indicator=True)   
relabeled_df['final_label'] = relabeled_df['labels_sorted1'].fillna(relabeled_df['label'])
#relabeled_df['name'] = relabeled_df['name'].astype(str) + '-CAR' #added for this version for our analysis

In [77]:
display(relabeled_df)

,key_0,Unnamed: 0,name,x,y,z,index,label,labels_sorted1,percent_assigned1,_merge,final_label
0,LDA1,0,LDA1,85.1479,129.4360,74.9584,0.0,EmptyLabel,NaN,NaN,left_only,EmptyLabel
1,LDA2,1,LDA2,80.4507,128.0110,73.9757,17.0,left hippocampus,NaN,NaN,left_only,left hippocampus
2,LDA3,2,LDA3,75.1392,127.7570,73.8188,17.0,left hippocampus,NaN,NaN,left_only,left hippocampus
3,LDA4,3,LDA4,70.4420,126.3330,72.8361,17.0,left hippocampus,NaN,NaN,left_only,left hippocampus
4,LDA5,4,LDA5,65.1305,126.0790,72.6792,0.0,EmptyLabel,NaN,NaN,left_only,EmptyLabel
...,...,...,...,...,...,...,...,...,...,...,...,...
111,LGR60,111,LGR60,30.5581,117.5990,83.7255,0.0,EmptyLabel,NaN,NaN,left_only,EmptyLabel
112,LGR61,112,LGR61,29.8546,107.8410,87.0782,0.0,EmptyLabel,NaN,NaN,left_only,EmptyLabel
113,LGR62,113,LGR62,31.2757,98.1849,90.4937,1015.0,left middle temporal,NaN,NaN,left_only,left middle temporal
114,LGR63,114,LGR63,31.2763,87.3494,89.9787,1015.0,left middle temporal,NaN,NaN,left_only,left middle temporal


In [78]:
relabeled_df = relabeled_df[['name', 'x', 'y', 'z', 'label', 'labels_sorted1', 'percent_assigned1', 'final_label']]
relabeled_df = relabeled_df.rename(columns={'final_label': 'threshold_label'})

relabeled_df contains the OG formula to take the 0.2 overlap labels

In [79]:
atropos_labels = atropos_labels.rename(columns={'labels_sorted': 'AT_labels', 'percent_assigned':'AT_perc'})

#MERGE ATROPOS AND DKT
merge_keys = workinglabels.merge(atropos_labels[['AT_labels', 'AT_perc']], left_on = workinglabels['name'], right_on= atropos_labels['name'], how='left')

In [81]:
empty_mergekeys = (merge_keys[merge_keys['label'] == 'EmptyLabel'])
empty_mergekeys = unnesting(empty_mergekeys, ['labels_sorted', 'percent_assigned','AT_labels','AT_perc'], axis=0)

def gray_labels(row):
    '''
    function looks for gray matter, if it finds it then it will look to return the label in which it is not Empty or None
    '''
    if (row['AT_labels0'] == 'gray matter') | (row['AT_labels1'] == 'gray matter') | (row['AT_labels2'] == 'gray matter'): #change this to 3 == stATEMENTS    MY GROUPING IS INCORRECT
        if (row['labels_sorted1'] != "EmptyLabel") | (row['labels_sorted1'] != None):
            return row['labels_sorted1']
        
        elif (row['labels_sorted2'] != "EmptyLabel") | (row['labels_sorted2'] != None):
            return row['labels_sorted2']
        
        elif (row['labels_sorted3'] != "EmptyLabel") | (row['labels_sorted3'] != None):
            return row['labels_sorted3']
        
        elif (row['labels_sorted4'] != "EmptyLabel") | (row['labels_sorted1'] != None):
            return row['labels_sorted4']
    
empty_mergekeys['at_dkt_labels'] = empty_mergekeys.apply(gray_labels, axis=1)
display(empty_mergekeys)

,labels_sorted0,labels_sorted1,labels_sorted2,labels_sorted3,percent_assigned0,percent_assigned1,percent_assigned2,percent_assigned3,AT_labels0,AT_labels1,...,AT_perc1,AT_perc2,key_0,name,x,y,z,index,label,at_dkt_labels
0,EmptyLabel,None,None,None,1.000000,NaN,NaN,NaN,gray matter,CSF,...,0.393939,NaN,LDA1,LDA1,85.1479,129.4360,74.9584,0,EmptyLabel,None
4,EmptyLabel,left hippocampus,None,None,0.878788,0.121212,NaN,NaN,CSF,gray matter,...,0.393939,0.030303,LDA5,LDA5,65.1305,126.0790,72.6792,0,EmptyLabel,left hippocampus
5,EmptyLabel,None,None,None,1.000000,NaN,NaN,NaN,white matter,gray matter,...,0.303030,NaN,LDA6,LDA6,59.9022,124.6290,71.6808,0,EmptyLabel,None
6,EmptyLabel,None,None,None,1.000000,NaN,NaN,NaN,white matter,None,...,NaN,NaN,LDA7,LDA7,55.1447,123.8750,71.6136,0,EmptyLabel,None
7,EmptyLabel,None,None,None,1.000000,NaN,NaN,NaN,white matter,None,...,NaN,NaN,LDA8,LDA8,49.8707,123.4770,70.4673,0,EmptyLabel,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,EmptyLabel,left superior temporal,None,None,0.939394,0.060606,NaN,NaN,CSF,gray matter,...,0.181818,NaN,LGR56,LGR56,32.8156,79.3049,104.2930,0,EmptyLabel,left superior temporal
108,EmptyLabel,left superior temporal,None,None,0.727273,0.272727,NaN,NaN,CSF,gray matter,...,0.151515,NaN,LGR57,LGR57,43.1415,147.9570,77.9387,0,EmptyLabel,left superior temporal
111,EmptyLabel,left superior temporal,None,None,0.969697,0.030303,NaN,NaN,CSF,gray matter,...,0.303030,NaN,LGR60,LGR60,30.5581,117.5990,83.7255,0,EmptyLabel,left superior temporal
112,EmptyLabel,left superior temporal,left middle temporal,None,0.848485,0.121212,0.030303,NaN,CSF,gray matter,...,0.151515,0.060606,LGR61,LGR61,29.8546,107.8410,87.0782,0,EmptyLabel,left superior temporal


In [82]:
#merge relabeled_df and empty_mergekeys by name, then add the at_dkt_labels column to the relabeled_df:
relabeled_df_2 = relabeled_df.merge(empty_mergekeys['at_dkt_labels'], left_on=relabeled_df['name'], right_on=empty_mergekeys['name'], how = 'left', indicator='merge_indicator')


In [83]:

relabeled_df_2['final_label'] = relabeled_df_2['at_dkt_labels'].fillna(relabeled_df_2['threshold_label'])

In [86]:
relabeled_df

,name,x,y,z,label,labels_sorted1,percent_assigned1,threshold_label
0,LDA1,85.1479,129.4360,74.9584,EmptyLabel,NaN,NaN,EmptyLabel
1,LDA2,80.4507,128.0110,73.9757,left hippocampus,NaN,NaN,left hippocampus
2,LDA3,75.1392,127.7570,73.8188,left hippocampus,NaN,NaN,left hippocampus
3,LDA4,70.4420,126.3330,72.8361,left hippocampus,NaN,NaN,left hippocampus
4,LDA5,65.1305,126.0790,72.6792,EmptyLabel,NaN,NaN,EmptyLabel
...,...,...,...,...,...,...,...,...
111,LGR60,30.5581,117.5990,83.7255,EmptyLabel,NaN,NaN,EmptyLabel
112,LGR61,29.8546,107.8410,87.0782,EmptyLabel,NaN,NaN,EmptyLabel
113,LGR62,31.2757,98.1849,90.4937,left middle temporal,NaN,NaN,left middle temporal
114,LGR63,31.2763,87.3494,89.9787,left middle temporal,NaN,NaN,left middle temporal


In [85]:
relabeled_df_2

,key_0,name,x,y,z,label,labels_sorted1,percent_assigned1,threshold_label,at_dkt_labels,merge_indicator,final_label
0,LDA1,LDA1,85.1479,129.4360,74.9584,EmptyLabel,NaN,NaN,EmptyLabel,None,both,EmptyLabel
1,LDA2,LDA2,80.4507,128.0110,73.9757,left hippocampus,NaN,NaN,left hippocampus,NaN,left_only,left hippocampus
2,LDA3,LDA3,75.1392,127.7570,73.8188,left hippocampus,NaN,NaN,left hippocampus,NaN,left_only,left hippocampus
3,LDA4,LDA4,70.4420,126.3330,72.8361,left hippocampus,NaN,NaN,left hippocampus,NaN,left_only,left hippocampus
4,LDA5,LDA5,65.1305,126.0790,72.6792,EmptyLabel,NaN,NaN,EmptyLabel,left hippocampus,both,left hippocampus
...,...,...,...,...,...,...,...,...,...,...,...,...
111,LGR60,LGR60,30.5581,117.5990,83.7255,EmptyLabel,NaN,NaN,EmptyLabel,left superior temporal,both,left superior temporal
112,LGR61,LGR61,29.8546,107.8410,87.0782,EmptyLabel,NaN,NaN,EmptyLabel,left superior temporal,both,left superior temporal
113,LGR62,LGR62,31.2757,98.1849,90.4937,left middle temporal,NaN,NaN,left middle temporal,NaN,left_only,left middle temporal
114,LGR63,LGR63,31.2763,87.3494,89.9787,left middle temporal,NaN,NaN,left middle temporal,NaN,left_only,left middle temporal


New code blocks that use gray matter to detect missing labels, should help a little.

In [89]:
def gray_labels(row):
    '''
    function looks for gray matter, if it finds it then it will look to return the label in which it is not Empty or None
    '''
    if (row['AT_labels0'] == 'gray matter') | (row['AT_labels1'] == 'gray matter') | (row['AT_labels2'] == 'gray matter'): #change this to 3 == stATEMENTS    MY GROUPING IS INCORRECT
        if (row['labels_sorted1'] != "EmptyLabel") | (row['labels_sorted1'] != None):
            return row['labels_sorted1']
        
        elif (row['labels_sorted2'] != "EmptyLabel") | (row['labels_sorted2'] != None):
            return row['labels_sorted2']
        
        elif (row['labels_sorted3'] != "EmptyLabel") | (row['labels_sorted3'] != None):
            return row['labels_sorted3']
        
        elif (row['labels_sorted4'] != "EmptyLabel") | (row['labels_sorted1'] != None):
            return row['labels_sorted4']
    

def label_fix(pt, data_directory, threshold = 0.20):
    '''
    label_fix reassigns labels overlapping brain regions to "empty labels" in our DKTantspynet output from IEEG_recon
    input:  pt - name of patient. example: 'HUP100' 
            data_directory - directory containing CNT_iEEG_BIGS folder. (must end in '/')
            threshold - arbitrary threshold that r=2mm surround of electrode must overlap with a brain region. default: threshold = 25%, Brain region has a 25% or more overlap.
    output: relabeled_df - a dataframe that contains 2 extra columns showing the second most overlapping brain region and the percent of overlap. 
    '''

    rid, brain_df = load_rid_forjson(pt, data_directory)
    if path.exists(data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-research3T_space-T00mri_atlas-DKTantspynet_radius-2_desc-vox_coordinates.json'.format(rid,rid)) == True:
        json_labels = data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-research3T_space-T00mri_atlas-DKTantspynet_radius-2_desc-vox_coordinates.json'.format(rid,rid)
    if path.exists(data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-implant01_space-T00mri_atlas-DKTantspynet_radius-2_desc-vox_coordinates.json'.format(rid,rid)) == True:
        json_labels = data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-implant01_space-T00mri_atlas-DKTantspynet_radius-2_desc-vox_coordinates.json'.format(rid,rid)
    if path.exists(data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-clinical01_space-T00mri_atlas-DKTantspynet_radius-2_desc-vox_coordinates.json'.format(rid,rid)) == True:
        json_labels = data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-clinical01_space-T00mri_atlas-DKTantspynet_radius-2_desc-vox_coordinates.json'.format(rid,rid)
        
    workinglabels = pd.read_json(json_labels, lines=True)
    empty = (workinglabels[workinglabels['label'] == 'EmptyLabel'])
    empty = unnesting(empty, ['labels_sorted', 'percent_assigned'], axis=0)
    empty = empty[np.isnan(empty['percent_assigned1']) == False]
    changed = empty[empty['percent_assigned1'] >= threshold]
    
    brain_df['name'] = brain_df['name'].str.replace('-CAR','')
    relabeled_df = brain_df.merge(changed[['labels_sorted1', 'percent_assigned1']], left_on=brain_df['name'], right_on=changed['name'], how = 'left', indicator=True)   
    relabeled_df['final_label'] = relabeled_df['labels_sorted1'].fillna(relabeled_df['label'])
    
    relabeled_df = relabeled_df[['name', 'x', 'y', 'z', 'label', 'labels_sorted1', 'percent_assigned1', 'final_label']]
    relabeled_df = relabeled_df.rename(columns={'final_label': 'threshold_label'})

    #load atropos
    if path.exists(data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-research3T_space-T00mri_atlas-atropos_radius-2_desc-vox_coordinates.json'.format(rid,rid)) == True:
        at_json_labels = data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-research3T_space-T00mri_atlas-atropos_radius-2_desc-vox_coordinates.json'.format(rid,rid)
    elif path.exists(data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-implant01_space-T00mri_atlas-atropos_radius-2_desc-vox_coordinates.json'.format(rid,rid)) == True:
        at_json_labels = data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-implant01_space-T00mri_atlas-atropos_radius-2_desc-vox_coordinates.json'.format(rid,rid)
    elif path.exists(data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-clinical01_space-T00mri_atlas-atropos_radius-2_desc-vox_coordinates.json'.format(rid,rid)) == True:
        at_json_labels = data_directory[1] + '/CNT_iEEG_BIDS/{}/derivatives/ieeg_recon/module3/{}_ses-clinical01_space-T00mri_atlas-atropos_radius-2_desc-vox_coordinates.json'.format(rid,rid)

    atropos_labels = pd.read_json(at_json_labels, lines = True)    
    atropos_labels = atropos_labels.rename(columns={'labels_sorted': 'AT_labels', 'percent_assigned':'AT_perc'})
    #MERGE ATROPOS AND DKT
    merge_keys = workinglabels.merge(atropos_labels[['AT_labels', 'AT_perc']], left_on = workinglabels['name'], right_on= atropos_labels['name'], how='left')
    
    empty_mergekeys = (merge_keys[merge_keys['label'] == 'EmptyLabel'])
    empty_mergekeys = unnesting(empty_mergekeys, ['labels_sorted', 'percent_assigned','AT_labels','AT_perc'], axis=0)
    empty_mergekeys['at_dkt_labels'] = empty_mergekeys.apply(gray_labels, axis=1)
    relabeled_df_2 = relabeled_df.merge(empty_mergekeys['at_dkt_labels'], left_on=relabeled_df['name'], right_on=empty_mergekeys['name'], how = 'left', indicator='merge_indicator')
    relabeled_df_2['final_label'] = relabeled_df_2['at_dkt_labels'].fillna(relabeled_df_2['threshold_label'])

    relabeled_df_2['name'] = relabeled_df_2['name'].astype(str) + '-CAR' #added for this version for our analysis

    return relabeled_df_2

In [90]:
df = label_fix('HUP142', data_directory, threshold = 0.20)
display(df)

,key_0,name,x,y,z,label,labels_sorted1,percent_assigned1,threshold_label,at_dkt_labels,merge_indicator,final_label
0,LDA1,LDA1-CAR,85.1479,129.4360,74.9584,EmptyLabel,NaN,NaN,EmptyLabel,None,both,EmptyLabel
1,LDA2,LDA2-CAR,80.4507,128.0110,73.9757,left hippocampus,NaN,NaN,left hippocampus,NaN,left_only,left hippocampus
2,LDA3,LDA3-CAR,75.1392,127.7570,73.8188,left hippocampus,NaN,NaN,left hippocampus,NaN,left_only,left hippocampus
3,LDA4,LDA4-CAR,70.4420,126.3330,72.8361,left hippocampus,NaN,NaN,left hippocampus,NaN,left_only,left hippocampus
4,LDA5,LDA5-CAR,65.1305,126.0790,72.6792,EmptyLabel,NaN,NaN,EmptyLabel,left hippocampus,both,left hippocampus
...,...,...,...,...,...,...,...,...,...,...,...,...
111,LGR60,LGR60-CAR,30.5581,117.5990,83.7255,EmptyLabel,NaN,NaN,EmptyLabel,left superior temporal,both,left superior temporal
112,LGR61,LGR61-CAR,29.8546,107.8410,87.0782,EmptyLabel,NaN,NaN,EmptyLabel,left superior temporal,both,left superior temporal
113,LGR62,LGR62-CAR,31.2757,98.1849,90.4937,left middle temporal,NaN,NaN,left middle temporal,NaN,left_only,left middle temporal
114,LGR63,LGR63-CAR,31.2763,87.3494,89.9787,left middle temporal,NaN,NaN,left middle temporal,NaN,left_only,left middle temporal


### EXTRA stuff idk what to do with

In [4]:

empty = (workinglabels[workinglabels['label'] == 'EmptyLabel'])
empty = unnesting(empty, ['labels_sorted', 'percent_assigned'], axis=0)
#empty = empty[np.isnan(empty['percent_assigned1']) == False]
changed = empty[empty['percent_assigned1'] >= threshold]

#brain_df['name'] = brain_df['name'].str.replace('-CAR','')
#relabeled_df = brain_df.merge(changed[['labels_sorted1', 'percent_assigned1']], left_on=brain_df['name'], right_on=changed['name'], how = 'left', indicator=True)
#relabeled_df['final_label'] = relabeled_df['labels_sorted1'].fillna(relabeled_df['label'])
#relabeled_df['name'] = relabeled_df['name'].astype(str) + '-CAR' #added for this version for our analysis

In [16]:
pd.options.display.max_rows = 999
empty = (workinglabels[workinglabels['label'] == 'EmptyLabel'])
empty = unnesting(empty, ['labels_sorted', 'percent_assigned'], axis=0)

at_empty = atropos_labels[(atropos_labels['label'] == 'white matter') | (atropos_labels['label'] == 'CSF') ]
at_empty = unnesting(at_empty, ['labels_sorted','percent_assigned'], axis = 0)

at_changed = at_empty[((at_empty['labels_sorted1'] == 'gray matter') & (at_empty['percent_assigned1'] >= at_threshold)) | ((at_empty['labels_sorted2'] == 'gray matter') & (at_empty['percent_assigned2'] >= at_threshold)) ]

changed = empty[((empty['labels_sorted1'] != 'EmptyLabel') & (empty['percent_assigned1'] >= threshold)) | ((empty['labels_sorted2'] != 'EmptyLabel') & (empty['percent_assigned2'] >= threshold/2)) ]

In [ ]:
#create func to parse to find the gray matter label
def GM_finder(row):
    if row['labels_sorted0'] == 'gray matter':
        return 'gray matter'
    elif row['labels_sorted1'] == 'gray matter':
        return 'gray matter'
    elif row['labels_sorted2'] == 'gray matter':
        return 'gray matter'
    else:
        return None

#create a label to merge on using other func.
at_changed['label_tochange_AT'] = at_changed.apply(GM_finder, axis = 1)

#create a label to merge on
changed['label_tochange'] = changed['labels_sorted1'].fillna(changed['labels_sorted2'])


In [31]:
brain_df['name'] = brain_df['name'].str.replace('-CAR','')
relabeled_df1 = brain_df.merge(changed['label_tochange'], left_on = brain_df['name'], right_on = changed['name'], how = 'left', indicator = True)
relabeled_df2 = brain_df.merge(at_changed['label_tochange_AT'], left_on = brain_df['name'], right_on = at_changed['name'], how = 'left', indicator = True)

